In [ ]:
!pip install langchain
!pip install typing-inspect==0.8.0 typing_extensions==4.5.0
!pip install pydantic==1.10.11
# !pip install llama-cpp-python
# !pip install tensorflow
!pip install accelerate
!pip install bitsandbytes
!pip install -U tokenizers
!pip install transformers
!pip install sentence-transformers

In [ ]:
%pip install ctransformers -q

In [ ]:
import glob
import pandas as pd
import sqlite3
import warnings
warnings.filterwarnings('ignore')
from langchain.llms import CTransformers
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler 

callback_manager=CallbackManager([StreamingStdOutCallbackHandler()])

config_sql = {'max_new_tokens': 512, 'repetition_penalty': 1.1,'temperature':0,'stop':[';']}

In [ ]:
file = "example.db"
try: 
    conn = sqlite3.connect(file) 
    print(f"Database {file} formed.") 
except: 
    print(f"Database {file} not formed.")

with sqlite3.connect(file) as conn:
    try:
        df.to_sql('df',conn,if_exists='replace',index=False)
        print('df table created')
    except:
        print('df table not created')
        
con = sqlite3.connect("example.db")
cursor = con.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

In [ ]:
def generate_sql_query(question):
    prompt=f"""
    {context}
    Question: Write a sql query for the question '{question}'
    Answer:
    """
    llm = CTransformers(model="TheBloke/OpenOrca-Platypus2-13B-GGUF", 
                        model_file="openorca-platypus2-13b.q4_K_M.gguf", 
                        model_type="llama", 
                        config=config_sql,callback_manager=callback_manager)
    return llm(prompt)

In [ ]:
question = ""
query_result = generate_sql_query(question)
with sqlite3.connect(file) as conn:
    df_result = pd.read_sql_query(query_result, conn)